In [30]:

import numpy as np
import torch


# load the data
X = np.load('data/x.npy')
y = np.load('data/y.npy')

# y = np.array([get_fen_score(board) for board in X])


# print(X[0])

# reshape y to be [n_samples, 1]
y = y.reshape(-1, 1)


# preview the data
print(X.shape)
print(y.shape)



# split the data into train and test
X_train = X[:int(len(X) * 0.8)]
y_train = y[:int(len(y) * 0.8)]
X_test = X[int(len(X) * 0.8):]
y_test = y[int(len(y) * 0.8):]

# convert to torch tensors
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()

X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()

(2030, 12, 8, 8)
(2030, 1)


In [31]:
# data preview
print(X_train.shape)
print(y_train.shape)



torch.Size([1624, 12, 8, 8])
torch.Size([1624, 1])


# only feed forward network

we define a network with some hidden layers and flattened inputs

In [41]:
import torch.functional as F
import torch
import torch.nn as nn

relu = F.F.relu


# define the neural network
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # fully connected layers
        self.fc1 = nn.Linear(12*8*8, 2048)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(2048, 1024)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(1024, 256)
        self.dropout3 = nn.Dropout(0.5)
        self.fc4 = nn.Linear(256, 128)
        self.dropout4 = nn.Dropout(0.5)
        self.fc5 = nn.Linear(128, 64)
        self.dropout5 = nn.Dropout(0.5)
        self.fc6 = nn.Linear(64, 32)
        self.dropout6 = nn.Dropout(0.5)
        self.fc7 = nn.Linear(32, 1)



    def forward(self, x):
        # reshape
        x = x.view(-1, 12*8*8)

        # forward pass
        x = relu(self.fc1(x))
        x = self.dropout1(x)
        x = relu(self.fc2(x))
        x = self.dropout2(x)
        x = relu(self.fc3(x))
        x = self.dropout3(x)
        x = relu(self.fc4(x))
        x = self.dropout4(x)
        x = relu(self.fc5(x))
        x = self.dropout5(x)
        x = relu(self.fc6(x))
        x = self.dropout6(x)
        x = self.fc7(x)

        return x
    
    def predict(self, x):
        # reshape
        x = x.view(-1, 12*8*8)

        # forward pass
        x = relu(self.fc1(x))
        x = relu(self.fc2(x))
        x = relu(self.fc3(x))
        x = relu(self.fc4(x))
        x = relu(self.fc5(x))
        x = relu(self.fc6(x))
        x = self.fc7(x)


        return x

# running the model

In [42]:
net = Net()

# define the loss function
criterion = torch.nn.L1Loss()
# criterion = torch.nn.L1Loss( reduction='mean', reduce=True)

# define the optimizer
optimizer = torch.optim.Adagrad(net.parameters(), lr=0.005)

for epoch in range(500):
    optimizer.zero_grad()
    outputs = net.forward(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    print('epoch {}, loss {}'.format(epoch, abs(loss.item())))

epoch 0, loss 2.493878126144409
epoch 1, loss 2.487050771713257
epoch 2, loss 2.4728353023529053
epoch 3, loss 2.3210716247558594
epoch 4, loss 1.956220030784607
epoch 5, loss 3.789656639099121
epoch 6, loss 2.47906494140625
epoch 7, loss 2.3724427223205566
epoch 8, loss 2.299419403076172
epoch 9, loss 2.1887967586517334
epoch 10, loss 2.044235944747925
epoch 11, loss 1.8599690198898315
epoch 12, loss 1.8163211345672607
epoch 13, loss 1.7528728246688843
epoch 14, loss 1.7443060874938965
epoch 15, loss 1.6942721605300903
epoch 16, loss 1.630506992340088
epoch 17, loss 1.7646198272705078
epoch 18, loss 1.6583354473114014
epoch 19, loss 1.544455885887146
epoch 20, loss 1.4772757291793823
epoch 21, loss 1.4265326261520386
epoch 22, loss 1.31303071975708
epoch 23, loss 1.3207076787948608
epoch 24, loss 1.273408055305481
epoch 25, loss 1.3079317808151245
epoch 26, loss 1.4173463582992554
epoch 27, loss 1.2798757553100586
epoch 28, loss 1.27120840549469
epoch 29, loss 1.288390040397644
epoch 

In [43]:
# test the model
with torch.no_grad():
    outputs = net.predict(X_test)
    loss = criterion(outputs, y_test)
    print('test loss: {}'.format(abs(loss.item())))

# save the model
MODEL_PATH = 'model_material.pth'
torch.save(net.state_dict(), MODEL_PATH)

test loss: 0.863443911075592


In [44]:
# load the model
net = Net()
net.load_state_dict(torch.load(MODEL_PATH))

# test the model
with torch.no_grad():
    outputs = net.predict(X_test)
    loss = criterion(outputs, y_test)
    print('test loss: {}'.format(abs(loss.item())))

test loss: 0.863443911075592
